In [1]:
# import packages we'll probably need
import pandas as pd
import numpy as np
import sys
import os

In [2]:
# start importing master data with the aisles dataset
df_aisles = pd.read_csv('aisles.csv')
df_aisles.head(3)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars


In [3]:
# then the departments dataset
df_departments = pd.read_csv('departments.csv')
df_departments.head(3)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery


In [4]:
# products dataset next
df_products = pd.read_csv('products.csv')
df_products.head(3)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7


In [5]:
# the products dataset has foreign keys for aisle and department names
# let's bring those into the products dataset
df_products = pd.merge(df_products, df_aisles, on='aisle_id').drop('aisle_id', axis=1)
df_products = pd.merge(df_products, df_departments, on='department_id').drop('department_id', axis=1)
df_products.head(3)

,product_id,product_name,aisle,department
0,1,Chocolate Sandwich Cookies,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,cookies cakes,snacks
2,102,Danish Butter Cookies,cookies cakes,snacks


In [6]:
# after working with the master data, let's bring in the transactional data
# starting with the orders dataset
# this dataset is the order header information
# we'll work with line items next, which is a lower level of detail
df_orders = pd.read_csv('orders.csv')
print(f'rows of data {len(df_orders)}')
df_orders.head(3)

rows of data 3421083


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0


In [7]:
# some of this data we can make more user friendly
# such as change day of week (order_dow) to have day names we recognize easier
# and changing the hour of day to a better time format: 8am, 6pm
# let's create some dictionaries to make this happen
# first day of the week
day_of_week_lookup = {
    'order_dow': [0, 1, 2, 3, 4, 5, 6],
    'day_of_week': ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']
}

# then hour of day
hour_of_day_lookup = {
    'order_hour_of_day': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
    'hour_of_day': ['12am', '1am', '2am', '3am', '4am', '5am', '6am',
                    '7am', '8am', '9am', '10am', '11am', '12pm', '1pm', 
                    '2pm', '3pm', '4pm', '5pm', '6pm', '7pm', '8pm', '9pm',
                    '10pm', '11pm']
}

In [8]:
# convert both of these to dataframes, then merge with the orders table
df_day_of_week = pd.DataFrame.from_dict(day_of_week_lookup)
df_hour_of_day = pd.DataFrame.from_dict(hour_of_day_lookup)

# then merge both onto orders dataset
df_orders = pd.merge(df_orders, df_day_of_week, on='order_dow').drop('order_dow', axis=1)
df_orders = pd.merge(df_orders, df_hour_of_day, on='order_hour_of_day').drop('order_hour_of_day', axis=1)
df_orders.head()

,order_id,user_id,eval_set,order_number,days_since_prior_order,day_of_week,hour_of_day
0,2539329,1,prior,1,NaN,Wed,8am
1,1273676,16,prior,1,NaN,Wed,8am
2,1560223,27,prior,68,4.0,Wed,8am
3,1837192,27,prior,80,6.0,Wed,8am
4,875452,29,prior,2,11.0,Wed,8am


In [9]:
# next let's get crazy
# there's no users master data table
# so let's use a random name generator to give our users names
# first let's pull out the distinct user ids into a list
user_ids = df_orders['user_id'].drop_duplicates().tolist()
len(user_ids)

206209

In [12]:
# 206,209 is quite a bit, let's create a random name
# for each of those - this takes like 30 mins to execute
# TODO: how do I speed this up?
import names

usernames = {}
for i in user_ids:
    usernames[i] = names.get_full_name()
    
# list(usernames.values())

In [13]:
# using these lists, we can make a dictionary
# then a lookup dataframe to merge back with the orders table
df_usernames = pd.DataFrame.from_dict({
    'user_id': user_ids,
    'usernames': list(usernames.values())
})

df_orders = pd.merge(df_orders, df_usernames, on='user_id').drop('user_id', axis=1)

In [14]:
df_orders.sort_values('order_id').head()

,order_id,eval_set,order_number,days_since_prior_order,day_of_week,hour_of_day,usernames
3041936,1,train,4,9.0,Fri,10am,Esmeralda Potvin
3152812,2,prior,3,8.0,Sat,9am,Jaime Somerville
952890,3,prior,16,12.0,Sat,5pm,Lillian Shilling
533525,4,prior,36,7.0,Tues,9am,Thomas Piserchio
2492142,5,prior,42,9.0,Sun,4pm,Howard Hall


In [15]:
# let's also add a row count column that can be easily aggregated
df_orders['count'] = 1
df_orders.head()

,order_id,eval_set,order_number,days_since_prior_order,day_of_week,hour_of_day,usernames,count
0,2539329,prior,1,NaN,Wed,8am,Susan Kelly,1
1,2550362,prior,10,30.0,Fri,8am,Susan Kelly,1
2,1187899,train,11,14.0,Fri,8am,Susan Kelly,1
3,3367565,prior,6,19.0,Wed,7am,Susan Kelly,1
4,2398795,prior,2,15.0,Thurs,7am,Susan Kelly,1


In [16]:
# save this dataset and bring into tableau
df_orders.to_csv('orders_cleansed.csv')